In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from simility_requests.return_rule_info_from_simility import ReturnRuleInfoFromSimility
from simility_requests.return_cassandra_dtypes import ReturnCassandraDatatypes
from rule_parsing.from_system_parser import FromSystemParser
from simility_apis.set_password import set_password
import pandas as pd
import numpy as np
import json

In [45]:
set_password()
params = {
    'url': 'https://app.simility.com',
    'app_prefix': 'republic_wireless',
    'user': 'james@simility.com',
    'entity': 'transaction'
}
# 088391CE2
# params = {
#     'url': 'https://eu.simility.com',
#     'app_prefix': 'omnyex',
#     'user': 'jlaidler@paypal.com',
#     'entity': 'orders'
# }
# params = {
#     'url': 'https://eu.simility.com',
#     'app_prefix': 'rebtel',
#     'user': 'james@simility.com',
#     'entity': 'orders'
# }
rri = ReturnRuleInfoFromSimility(**params)
rule_info = rri.request()

Please provide your password for logging into the Simility platform:  ········


# Get system rule conditions

In [46]:
conditions = rule_info['conditions'].apply(lambda x: json.loads(x)).to_dict()

In [47]:
params = {
    'url': 'https://app.simility.com',
    'app_prefix': 'republic_wireless',
    'user': 'james@simility.com',
    'base_entity': 'transaction'
}
# params = {
#     'url': 'https://eu.simility.com',
#     'app_prefix': 'omnyex',
#     'user': 'jlaidler@paypal.com',
#     'base_entity': 'orders'
# }
# params = {
#     'url': 'https://eu.simility.com',
#     'app_prefix': 'rebtel',
#     'user': 'james@simility.com',
#     'base_entity': 'orders'
# }

In [48]:
rcdt = ReturnCassandraDatatypes(**params)

In [49]:
rw_cass_dtypes = rcdt.request()

In [50]:
rw_cass_dtypes['CassandraDatatype'].unique()

array(['FLOAT', 'BIGINT', 'TEXT', 'MAP', 'INT', 'DOUBLE', 'SET',
       'BOOLEAN', 'TIMESTAMP', 'BLOB', 'LIST'], dtype=object)

In [51]:
rw_cass_dtypes.tail()

PipelineOutputFieldName  \
Entity      ReferenceField CassandraFieldName                             
transaction -              user_agent_os_family    user_agent_os_family   
                           user_agent_os_vendor    user_agent_os_vendor   
                           vpn_high_1m_ratio          vpn_high_1m_ratio   
                           vpn_high_1w_ratio          vpn_high_1w_ratio   
                           vpn_high_2d_ratio          vpn_high_2d_ratio   

                                                CassandraDatatype  
Entity      ReferenceField CassandraFieldName                      
transaction -              user_agent_os_family              TEXT  
                           user_agent_os_vendor              TEXT  
                           vpn_high_1m_ratio               DOUBLE  
                           vpn_high_1w_ratio               DOUBLE  
                           vpn_high_2d_ratio               DOUBLE

In [52]:
cassandra_field_names = rw_cass_dtypes.reset_index().apply(lambda x: x['CassandraFieldName'] if x['ReferenceField'] == '-' else f"{x['ReferenceField']}.{x['CassandraFieldName']}", axis=1)

In [53]:
cassandra_field_names = pd.concat([rw_cass_dtypes.reset_index()['PipelineOutputFieldName'], cassandra_field_names], axis=1).set_index('PipelineOutputFieldName').squeeze().to_dict()

# Convert to ARGO dict format 

In [54]:
fsp = FromSystemParser(system_dicts=conditions)

In [55]:
rule_dicts = fsp.parse()

/Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_parsing/rule_parsing/from_system_parser.py:76: UserWarning: ln_review_shipping_address_1 : Operator `java_el` is not currently supported in ARGO. Rule cannot be parsed.
  warnings.warn(f'{rule_name} : {e}')
/Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_parsing/rule_parsing/from_system_parser.py:76: UserWarning: ln_review_city : Operator `java_el` is not currently supported in ARGO. Rule cannot be parsed.
  warnings.warn(f'{rule_name} : {e}')
/Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_parsing/rule_parsing/from_system_parser.py:76: UserWarning: ln_review_order_items : Operator `java_el` is not currently supported in ARGO. Rule cannot be parsed.
  warnings.warn(f'{rule_name} : {e}')
/Users/jlaidler/Documents/tigress/tigress/argo/argo/rule_parsing/rule_parsing/from_system_parser.py:76: UserWarning: ln_escalate_shipping_address : Operator `java_el` is not currently supported in ARGO. Rule cannot be parsed.
  w

In [56]:
# [v for _, v in conditions.items() if "java" in str(v)]

In [57]:
# rule_dicts['IP in VE']

In [58]:
# conditions['IP in VE']

# Convert back to system format

In [59]:
from rule_parsing.to_system_parser import ToSystemParser

In [60]:
tsp = ToSystemParser(rule_dicts=rule_dicts, field_datatypes=dict(rw_cass_dtypes.values), cassandra_field_names=cassandra_field_names)

In [61]:
parsed_system_dicts = tsp.parse()

In [62]:
results = {}
for rule_name, rule_dict in conditions.items():
    if rule_name in parsed_system_dicts.keys():
        parsed_dict = parsed_system_dicts[rule_name]
        results[rule_name] = parsed_dict == rule_dict

In [63]:
sum(list(results.values()))

114

In [64]:
# results

In [65]:
# conditions['IP in VE']

In [66]:
# parsed_system_dicts['IP in VE']

In [67]:
# rule_dicts['NodeviceId_Web_20201101']

In [68]:
# conditions['NodeviceId_Web_20201101']

In [69]:
# parsed_system_dicts['NodeviceId_Web_20201101']

# Convert to ARGO rule dicts again, then apply to dataset

In [70]:
fsp_p = FromSystemParser(system_dicts=parsed_system_dicts)

In [71]:
parsed_rule_dicts = fsp_p.parse()

In [72]:
results = {}
mismatches = []
for rule_name, rule_dict in rule_dicts.items():
    parsed_dict = parsed_rule_dicts[rule_name]
    results[rule_name] = parsed_dict == rule_dict
    if parsed_dict != rule_dict:
        mismatches = rule_name

In [73]:
mismatches

[]

# Results

- RW all match
- Omnyex all match
- Rebtel all match

In [32]:
unit_test_rule_dicts = {'Rule1': {'condition': 'AND',
                            'rules': [{'condition': 'OR',
                                       'rules': [{'field': 'payer_id_sum_approved_txn_amt_per_paypalid_1day',
                                                  'operator': 'greater_or_equal',
                                                  'value': 60.0},
                                                 {'field': 'payer_id_sum_approved_txn_amt_per_paypalid_7day',
                                                  'operator': 'greater',
                                                  'value': 120.0},
                                                 {'field': 'payer_id_sum_approved_txn_amt_per_paypalid_30day',
                                                  'operator': 'less_or_equal',
                                                  'value': 500.0}]},
                                      {'field': 'num_items', 'operator': 'equal', 'value': 1.0}]},
                  'Rule2': {'condition': 'AND',
                            'rules': [{'field': 'ml_cc_v0', 'operator': 'less', 'value': 0.315},
                                      {'condition': 'OR',
                                       'rules': [{'field': 'method_clean',
                                                  'operator': 'equal',
                                                  'value': 'checkout'},
                                                 {'field': 'method_clean',
                                                  'operator': 'begins_with', 'value': 'checkout'},
                                                 {'field': 'method_clean',
                                                  'operator': 'ends_with', 'value': 'checkout'},
                                                 {'field': 'method_clean',
                                                  'operator': 'contains', 'value': 'checkout'},
                                                 {'field': 'ip_address',
                                                  'operator': 'is_not_null', 'value': None},
                                                 {'field': 'ip_isp', 'operator': 'is_not_empty', 'value': None}]}]},
                  'Rule3': {'condition': 'AND',
                            'rules': [{'field': 'method_clean',
                                       'operator': 'not_begins_with',
                                       'value': 'checkout'},
                                      {'field': 'method_clean',
                                       'operator': 'not_ends_with', 'value': 'checkout'},
                                      {'field': 'method_clean',
                                       'operator': 'not_contains', 'value': 'checkout'},
                                      {'condition': 'OR',
                                       'rules': [{'field': 'ip_address', 'operator': 'is_null', 'value': None},
                                                 {'field': 'ip_isp', 'operator': 'is_empty', 'value': None}]}]},
                  'Rule4': {'condition': 'AND',
                            'rules': [{'field': 'forwarder_address', 'operator': 'equal', 'value': True},
                                      {'field': 'is_shipping_billing_address_same',
                                       'operator': 'equal',
                                       'value': False}]},
                  'Rule5': {'condition': 'AND',
                            'rules': [{'field': 'ad_price_type',
                                       'operator': 'not_in',
                                       'value': ['FREE', 'NEGOTIATION']},
                                      {'field': 'ad_price_type', 'operator': 'in', 'value': ['FOO', 'BAR']}]},
                  'Rule6': {'condition': 'AND',
                            'rules': [{'field': 'ip_country_iso_code',
                                       'operator': 'equal_field',
                                       'value': 'billing_country'},
                                      {'field': 'country_id',
                                       'operator': 'not_equal_field',
                                       'value': 'ip_country_iso_code'}]}}

In [33]:
unit_test_system_dicts = {'Rule1': {'condition': 'AND',
                              'rules': [{'condition': 'OR',
                                         'rules': [{'id': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
                                                    'field': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
                                                    'type': 'double',
                                                    'operator': 'greater_or_equal',
                                                    'value': 'value.60.0'},
                                                   {'id': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
                                                    'field': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
                                                    'type': 'double',
                                                    'operator': 'greater',
                                                    'value': 'value.120.0'},
                                                   {'id': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
                                                    'field': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
                                                    'type': 'double',
                                                    'operator': 'less_or_equal',
                                                    'value': 'value.500.0'}],
                                         },
                                        {'id': 'num_items',
                                         'field': 'num_items',
                                         'type': 'integer',
                                         'operator': 'equal',
                                         'value': 'value.1'},
                                        ]},
                    'Rule2': {'condition': 'AND',
                                         'rules': [{'id': 'ml_cc_v0',
                                                    'field': 'ml_cc_v0',
                                                    'type': 'double',
                                                    'operator': 'less',
                                                    'value': 'value.0.315'},
                                                   {'condition': 'OR',
                                                    'rules': [{'id': 'method_clean',
                                                               'field': 'method_clean',
                                                               'type': 'string',
                                                               'operator': 'equal',
                                                               'value': 'value.checkout'},
                                                              {'id': 'method_clean',
                                                               'field': 'method_clean',
                                                               'type': 'string',
                                                               'operator': 'begins_with',
                                                               'value': 'value.checkout'},
                                                              {'id': 'method_clean',
                                                               'field': 'method_clean',
                                                               'type': 'string',
                                                               'operator': 'ends_with',
                                                               'value': 'value.checkout'},
                                                              {'id': 'method_clean',
                                                               'field': 'method_clean',
                                                               'type': 'string',
                                                               'operator': 'contains',
                                                               'value': 'value.checkout'},
                                                              {'id': 'ip_address',
                                                               'field': 'ip_address',
                                                               'type': 'string',
                                                               'operator': 'is_not_null',
                                                               'value': None},
                                                              {'id': 'ip_isp',
                                                               'field': 'ip_isp',
                                                               'type': 'string',
                                                               'operator': 'is_not_empty',
                                                               'value': None}
                                                              ]}]},
                    'Rule3': {'condition': 'AND',
                              'rules': [{'id': 'method_clean',
                                         'field': 'method_clean',
                                         'type': 'string',
                                         'operator': 'not_begins_with',
                                         'value': 'value.checkout'},
                                        {'id': 'method_clean',
                                         'field': 'method_clean',
                                         'type': 'string',
                                         'operator': 'not_ends_with',
                                         'value': 'value.checkout'},
                                        {'id': 'method_clean',
                                         'field': 'method_clean',
                                         'type': 'string',
                                         'operator': 'not_contains',
                                         'value': 'value.checkout'},
                                        {'condition': 'OR',
                                         'rules': [{'id': 'ip_address',
                                                    'field': 'ip_address',
                                                    'type': 'string',
                                                    'operator': 'is_null',
                                                    'value': None},
                                                   {'id': 'ip_isp',
                                                    'field': 'ip_isp',
                                                    'type': 'string',
                                                    'operator': 'is_empty',
                                                    'value': None},
                                                   ],
                                         }]},
                    'Rule4': {'condition': 'AND',
                              'rules': [{'id': 'forwarder_address',
                                         'field': 'forwarder_address',
                                         'type': 'boolean',
                                         'input': 'radio',
                                         'operator': 'equal',
                                         'value': '1'},
                                        {'id': 'is_shipping_billing_address_same',
                                         'field': 'is_shipping_billing_address_same',
                                         'type': 'boolean',
                                         'input': 'radio',
                                         'operator': 'equal',
                                         'value': '0'}]},
                    'Rule5': {'condition': 'AND',
                              'rules': [{'id': 'ad_price_type',
                                         'field': 'ad_price_type',
                                         'type': 'string',
                                         'operator': 'not_in',
                                         'value': 'value.FREE,NEGOTIATION'},
                                        {'id': 'ad_price_type',
                                         'field': 'ad_price_type',
                                         'type': 'string',
                                         'operator': 'in',
                                         'value': 'value.FOO,BAR'}
                                        ]},
                    'Rule6': {'condition': 'AND',
                              'rules': [{'id': 'ip_country_iso_code',
                                         'field': 'ip_country_iso_code',
                                         'type': 'string',
                                         'operator': 'equal',
                                         'value': 'field.billing_country',
                                         },
                                        {'id': 'country_id',
                                         'field': 'country_id',
                                         'type': 'string',
                                         'operator': 'not_equal',
                                         'value': 'field.ip_country_iso_code',
                                         }]}}

In [34]:
field_datatypes = {
    'payer_id_sum_approved_txn_amt_per_paypalid_1day': 'DOUBLE',
    'payer_id_sum_approved_txn_amt_per_paypalid_7day': 'DOUBLE',
    'payer_id_sum_approved_txn_amt_per_paypalid_30day': 'DOUBLE',
    'num_items': 'INT',
    'ml_cc_v0': 'DOUBLE',
    'method_clean': 'TEXT',
    'method_clean': 'TEXT',
    'method_clean': 'TEXT',
    'method_clean': 'TEXT',
    'ip_address': 'TEXT',
    'ip_isp': 'TEXT',
    'method_clean': 'TEXT',
    'method_clean': 'TEXT',
    'method_clean': 'TEXT',
    'ip_address': 'TEXT',
    'ip_isp': 'TEXT',
    'forwarder_address': 'BOOLEAN',
    'is_shipping_billing_address_same': 'BOOLEAN',
    'ad_price_type': 'TEXT',
    'ad_price_type': 'TEXT',
    'ip_country_iso_code': 'TEXT',
    'country_id': 'TEXT',
    'billing_country': 'TEXT'
}
cassandra_field_names = {
    'payer_id_sum_approved_txn_amt_per_paypalid_1day': 'payer_id.sum_approved_txn_amt_per_paypalid_1day',
    'payer_id_sum_approved_txn_amt_per_paypalid_7day': 'payer_id.sum_approved_txn_amt_per_paypalid_7day',
    'payer_id_sum_approved_txn_amt_per_paypalid_30day': 'payer_id.sum_approved_txn_amt_per_paypalid_30day',
    'num_items': 'num_items',
    'ml_cc_v0': 'ml_cc_v0',
    'method_clean': 'method_clean',
    'method_clean': 'method_clean',
    'method_clean': 'method_clean',
    'method_clean': 'method_clean',
    'ip_address': 'ip_address',
    'ip_isp': 'ip_isp',
    'method_clean': 'method_clean',
    'method_clean': 'method_clean',
    'method_clean': 'method_clean',
    'ip_address': 'ip_address',
    'ip_isp': 'ip_isp',
    'forwarder_address': 'forwarder_address',
    'is_shipping_billing_address_same': 'is_shipping_billing_address_same',
    'ad_price_type': 'ad_price_type',
    'ad_price_type': 'ad_price_type',
    'ip_country_iso_code': 'ip_country_iso_code',
    'country_id': 'country_id',
    'billing_country': 'billing_country'
}

In [35]:
# tsp = ToSystemParser(rule_dicts=unit_test_rule_dicts, field_datatypes=field_datatypes, cassandra_field_names=cassandra_field_names)

# parsed_system_dicts = tsp.parse()

# parsed_system_dicts == unit_test_system_dicts

# mismatches = []
# for rule_name, system_dict in unit_test_system_dicts.items():
#     parsed_dict = parsed_system_dicts[rule_name]  
#     if parsed_dict != system_dict:
#         mismatches.append(rule_name)

# mismatches

In [36]:
from rules.rules import Rules

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
r = Rules(system_dicts=unit_test_system_dicts)

In [39]:
r.as_rule_dicts()

{'Rule1': {'condition': 'AND',
  'rules': [{'condition': 'OR',
    'rules': [{'field': 'payer_id_sum_approved_txn_amt_per_paypalid_1day',
      'operator': 'greater_or_equal',
      'value': 60.0},
     {'field': 'payer_id_sum_approved_txn_amt_per_paypalid_7day',
      'operator': 'greater',
      'value': 120.0},
     {'field': 'payer_id_sum_approved_txn_amt_per_paypalid_30day',
      'operator': 'less_or_equal',
      'value': 500.0}]},
   {'field': 'num_items', 'operator': 'equal', 'value': 1.0}]},
 'Rule2': {'condition': 'AND',
  'rules': [{'field': 'ml_cc_v0', 'operator': 'less', 'value': 0.315},
   {'condition': 'OR',
    'rules': [{'field': 'method_clean',
      'operator': 'equal',
      'value': 'checkout'},
     {'field': 'method_clean', 'operator': 'begins_with', 'value': 'checkout'},
     {'field': 'method_clean', 'operator': 'ends_with', 'value': 'checkout'},
     {'field': 'method_clean', 'operator': 'contains', 'value': 'checkout'},
     {'field': 'ip_address', 'operator

In [41]:
r.as_rule_strings(as_numpy=True)

{'Rule1': "((X['payer_id_sum_approved_txn_amt_per_paypalid_1day'].to_numpy(na_value=np.nan)>=60.0)|(X['payer_id_sum_approved_txn_amt_per_paypalid_7day'].to_numpy(na_value=np.nan)>120.0)|(X['payer_id_sum_approved_txn_amt_per_paypalid_30day'].to_numpy(na_value=np.nan)<=500.0))&(X['num_items'].to_numpy(na_value=np.nan)==1.0)",
 'Rule2': "(X['ml_cc_v0'].to_numpy(na_value=np.nan)<0.315)&((X['method_clean'].to_numpy(na_value=np.nan)=='checkout')|(X['method_clean'].str.startswith('checkout', na=False))|(X['method_clean'].str.endswith('checkout', na=False))|(X['method_clean'].str.contains('checkout', na=False))|(~pd.isna(X['ip_address'].to_numpy(na_value=np.nan)))|(X['ip_isp'].fillna('')!=''))",
 'Rule3': "(~X['method_clean'].str.startswith('checkout', na=False))&(~X['method_clean'].str.endswith('checkout', na=False))&(~X['method_clean'].str.contains('checkout', na=False))&((pd.isna(X['ip_address'].to_numpy(na_value=np.nan)))|(X['ip_isp'].fillna('')==''))",
 'Rule4': "(X['forwarder_address'].t

In [28]:
unit_test_system_dicts == r.system_dicts

True